In [1]:
import pandas as pd
import numpy as np

In [2]:
import requests

In [3]:
import json

In [4]:
india_district_url = "https://api.covid19india.org/v2/state_district_wise.json"
india_states_daily_url = "https://api.covid19india.org/states_daily.json"

In [5]:
dict_dist = requests.get(india_district_url).json()
dict_states = requests.get(india_states_daily_url).json()

In [11]:
df_dist = pd.DataFrame(dict_dist)
df_states = pd.DataFrame(dict_states['states_daily'])

In [12]:
# df_states['date'] = pd.to_datetime(df_states['date'])
dfsc = df_states[df_states.status == 'Confirmed'].set_index('date').T
dfsr = df_states[df_states.status == 'Recovered'].set_index('date').T
dfsd = df_states[df_states.status == 'Deceased'].set_index('date').T

In [13]:
dfsc.reset_index().head()

date,index,14-Mar-20,15-Mar-20,16-Mar-20,17-Mar-20,18-Mar-20,19-Mar-20,20-Mar-20,21-Mar-20,22-Mar-20,...,11-May-20,12-May-20,13-May-20,14-May-20,15-May-20,16-May-20,17-May-20,18-May-20,19-May-20,20-May-20
0,an,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ap,1,0,0,0,0,2,0,2,1,...,38,71,48,68,102,48,25,52,57,71
2,ar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,as,0,0,0,0,0,0,0,0,0,...,2,0,15,7,3,3,8,15,42,31
4,br,0,0,0,0,0,0,0,0,2,...,42,130,74,46,34,145,142,103,96,88


In [14]:
df_states

,an,ap,ar,as,br,ch,ct,date,dd,dl,...,sk,status,tg,tn,tr,tt,un,up,ut,wb
0,0,1,0,0,0,0,0,14-Mar-20,0,7,...,0,Confirmed,1,1,0,81,0,12,0,0
1,0,0,0,0,0,0,0,14-Mar-20,0,1,...,0,Recovered,0,0,0,9,0,4,0,0
2,0,0,0,0,0,0,0,14-Mar-20,0,1,...,0,Deceased,0,0,0,2,0,0,0,0
3,0,0,0,0,0,0,0,15-Mar-20,0,0,...,0,Confirmed,2,0,0,27,0,1,0,0
4,0,0,0,0,0,0,0,15-Mar-20,0,1,...,0,Recovered,1,0,0,4,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,0,69,0,0,0,3,0,19-May-20,0,265,...,0,Recovered,9,489,27,3032,0,135,0,68
200,0,2,0,0,0,0,0,19-May-20,0,6,...,0,Deceased,4,3,0,146,0,5,0,6
201,0,71,0,31,88,3,14,20-May-20,0,534,...,0,Confirmed,27,743,0,5543,282,249,11,142
202,0,43,0,7,54,79,0,20-May-20,0,442,...,0,Recovered,2,987,0,3099,0,148,1,62


In [232]:
# OVERALL DATA FOR STATES
df_states_drop = df_states.drop(['date','status'],axis = 1)
df_states_drop = df_states_drop.replace('',0).astype(int)
df_states_status = df_states.status

In [254]:
df_states_last = df_states[-3:].drop('date', axis = 1)
states_cols = df_states_last.columns
df_states_last = df_states_last.set_index('status').replace('',0).astype(int).T

In [234]:
df_states_status = df_states.status
df_states_comb = pd.concat([df_states_drop,df_states_status], axis = 1)

In [278]:
df_states_grp = df_states_comb.groupby('status', as_index = False).agg(sum)
df_states_grp = df_states_grp[states_cols].set_index('status').T
states_grp_cols = df_states_grp.columns
df_states_last = df_states_last[states_grp_cols]

In [194]:
df_dist['state'] = df_dist['state'].str.lower()
df_dist['statecode'] = df_dist['statecode'].str.lower()
df_dist_sel = df_dist[['state','statecode']].set_index('statecode')

In [325]:
df_dist.districtData[df_dist.statecode == 'AN'].values[0]

[{'district': 'North and Middle Andaman',
  'notes': '',
  'active': 0,
  'confirmed': 1,
  'deceased': 0,
  'recovered': 1,
  'delta': {'confirmed': 0, 'deceased': 0, 'recovered': 0}},
 {'district': 'South Andaman',
  'notes': '',
  'active': 18,
  'confirmed': 32,
  'deceased': 0,
  'recovered': 14,
  'delta': {'confirmed': 0, 'deceased': 0, 'recovered': 4}}]

In [333]:
# DISTRICT DATA
district_data_list = df_dist.districtData[df_dist.statecode == 'AN'].values[0]

In [337]:
district_data_list_new = []
for dct in district_data_list:
    dist_dict = {}
    dist_dict['district'] = dct['district']
    dist_dict['confirmed'] = dct['confirmed']
    dist_dict['deaths'] = dct['deceased']
    dist_dict['recovered'] = dct['recovered']
    dist_dict['confirmed_incr'] = dct['delta']['confirmed']
    dist_dict['deaths_incr'] = dct['delta']['deceased']
    dist_dict['recovered_incr'] = dct['delta']['recovered']
    dist_dict['confirmed_incr_rate'] = 0.0 if dist_dict['confirmed'] - dist_dict['confirmed_incr'] == 0 else 100 * dist_dict['confirmed_incr'] / (dist_dict['confirmed'] - dist_dict['confirmed_incr'])
    dist_dict['deaths_incr_rate'] = 0.0 if dist_dict['deaths'] - dist_dict['deaths_incr'] == 0 else 100 * dist_dict['deaths_incr'] / (dist_dict['deaths'] - dist_dict['deaths_incr'])
    dist_dict['recovered_incr_rate'] = 0.0 if dist_dict['recovered'] - dist_dict['recovered_incr'] == 0 else 100 * dist_dict['recovered_incr'] / (dist_dict['recovered'] - dist_dict['recovered_incr'])
    dist_dict['confirmed_incr_rate'] = str(dist_dict['confirmed_incr_rate']) + '%'
    dist_dict['deaths_incr_rate'] = str(dist_dict['deaths_incr_rate']) + '%'
    dist_dict['recovered_incr_rate'] = str(dist_dict['recovered_incr_rate']) + '%'
    
    district_data_list_new.append(dist_dict)

In [339]:
len(district_data_list_new)

2

In [217]:
state_code_dict = {
        "state_code":["TT","MH","GJ","DL","RJ","MP","TN","UP","AP","TG","WB","JK","KA","KL","PB","HR","BR","OR","JH","UT","HP","CT","AS","CH","AN","LA","ML","PY","GA","MN","TR","MZ","AR","NL","DN","DD","LD","SK"],
        "state":["Total","Maharashtra","Gujarat","Delhi","Rajasthan","Madhya Pradesh","Tamil Nadu","Uttar Pradesh","Andhra Pradesh","Telangana","West Bengal","Jammu and Kashmir","Karnataka","Kerala","Punjab","Haryana","Bihar","Odisha","Jharkhand","Uttarakhand","Himachal Pradesh","Chhattisgarh","Assam","Chandigarh","Andaman and Nicobar Islands","Ladakh","Meghalaya","Puducherry","Goa","Manipur","Tripura","Mizoram","Arunachal Pradesh","Nagaland","Dadra and Nagar Haveli","Daman and Diu","Lakshadweep","Sikkim"]
        }

In [243]:
state_code_df = pd.DataFrame(state_code_dict).sort_values('state_code')
state_code_df['state_code'] = state_code_df['state_code'].str.lower()
state_code_df.set_index('state_code', inplace = True)

In [315]:
state_code_df = pd.read_csv("state_coords.csv")
state_code_df['state_code'] = state_code_df['state_code'].str.lower()
state_code_df.set_index('state_code', inplace = True)

In [316]:
state_code_df.head()

,state,latitude,longitude
state_code,,,
an,Andaman and Nicobar Islands,11.740087,92.658640
ap,Andhra Pradesh,15.912900,79.740000
ar,Arunachal Pradesh,28.210000,94.720000
as,Assam,26.244156,92.537842
br,Bihar,25.090000,85.310000


In [321]:
state_code_df_mod = state_code_df.copy()
state_code_df_mod['state'] = state_code_df_mod['state'].str.lower()
state_code_df_filt = state_code_df_mod[state_code_df_mod.state == 'bihar']

In [322]:
float(state_code_df_filt.latitude.values[0])

25.09

In [323]:
state_code_df_filt.index[0]

'br'

In [320]:
state_code_df[state_code_df.state == "Bihar"].latitude.values[0]

25.09

In [281]:
df_states_overall = pd.concat([state_code_df,df_states_grp,df_states_last], axis = 1).reset_index()

In [282]:
df_states_overall.columns = ["state_code","state","confirmed", "deaths", "recovered","confirmed_incr","deaths_incr","recovered_incr"]

In [283]:
df_states_overall.tail(10)

,state_code,state,confirmed,deaths,recovered,confirmed_incr,deaths_incr,recovered_incr
28,py,Puducherry,8,0,5,0,0,1
29,rj,Rajasthan,2262,50,744,77,9,115
30,sk,Sikkim,0,0,0,0,0,0
31,tg,Telangana,1003,25,332,2,0,16
32,tn,Tamil Nadu,1937,24,1101,52,0,81
33,tr,Tripura,2,0,2,0,0,0
34,tt,Total,29496,936,7108,1607,56,585
35,up,Uttar Pradesh,1986,31,399,113,1,72
36,ut,Uttarakhand,51,0,33,0,0,7
37,wb,West Bengal,649,20,105,38,0,0


In [284]:
df_states_overall["confirmed_incr_rate"] = np.round(100 * df_states_overall["confirmed_incr"] / (df_states_overall["confirmed"] - df_states_overall["confirmed_incr"]))
df_states_overall["deaths_incr_rate"] = np.round(100 * df_states_overall["deaths_incr"] / (df_states_overall["deaths"] - df_states_overall["deaths_incr"]))
df_states_overall["recovered_incr_rate"] = np.round(100 * df_states_overall["recovered_incr"] / (df_states_overall["recovered"] - df_states_overall["recovered_incr"]))

df_states_overall.fillna(0, inplace = True)
df_states_overall["deaths_incr_rate"] = df_states_overall["deaths_incr_rate"].astype(str) + '%'
df_states_overall["confirmed_incr_rate"] = df_states_overall["confirmed_incr_rate"].astype(str) + '%'
df_states_overall["recovered_incr_rate"] = df_states_overall["recovered_incr_rate"].astype(str) + '%'    

In [120]:
state_code = df_dist.statecode[df_dist.state == 'Maharashtra'].values[0].lower()

In [301]:
def get_state_data(state_code):
    
    dfsc_filt = dfsc.loc[state_code]
    dfsd_filt = dfsd.loc[state_code]
    dfsr_filt = dfsr.loc[state_code]
    
    # FINDING IMPORTANT DATA
    state_df = pd.concat([dfsc_filt, dfsd_filt, dfsr_filt], axis = 1)
    state_df = state_df.astype(int)
    state_df.columns = ['confirmed_incr', 'deaths_incr', 'recovered_incr']
    state_df['confirmed'] = state_df['confirmed_incr'].cumsum()
    state_df['deaths'] = state_df['deaths_incr'].cumsum()
    state_df['recovered'] = state_df['recovered_incr'].cumsum()
    
    state_df["deaths_incr_rate"] = np.round(100 * state_df["deaths_incr"] / (state_df["deaths"] - state_df["deaths_incr"]))
    state_df["confirmed_incr_rate"] = np.round(100 * state_df["confirmed_incr"] / (state_df["confirmed"] - state_df["confirmed_incr"]))
    state_df["recovered_incr_rate"] = np.round(100 * state_df["recovered_incr"] / (state_df["recovered"] - state_df["recovered_incr"]))
   
    # CLEANING NA DATA
    state_df = state_df.replace([np.inf, -np.inf], np.nan)
    state_df.fillna(0, inplace = True)
    
    state_df["deaths_incr_rate"] = state_df["deaths_incr_rate"].astype(str) + '%'
    state_df["confirmed_incr_rate"] = state_df["confirmed_incr_rate"].astype(str) + '%'
    state_df["recovered_incr_rate"] = state_df["recovered_incr_rate"].astype(str) + '%'
    
    return state_df

In [303]:
state_df = get_state_data(state_code)

In [304]:
state_df

,confirmed_incr,deaths_incr,recovered_incr,confirmed,deaths,recovered,deaths_incr_rate,confirmed_incr_rate,recovered_incr_rate
date,,,,,,,,,
2020-03-14,14,0,0,14,0,0,0.0%,0.0%,0.0%
2020-03-15,18,0,0,32,0,0,0.0%,129.0%,0.0%
2020-03-16,6,0,0,38,0,0,0.0%,19.0%,0.0%
2020-03-17,3,1,0,41,1,0,0.0%,8.0%,0.0%
2020-03-18,3,0,0,44,1,0,0.0%,7.0%,0.0%
2020-03-19,4,0,0,48,1,0,0.0%,9.0%,0.0%
2020-03-20,4,0,0,52,1,0,0.0%,8.0%,0.0%
2020-03-21,12,0,0,64,1,0,0.0%,23.0%,0.0%
2020-03-22,10,1,0,74,2,0,100.0%,16.0%,0.0%


In [305]:
start_date = pd.to_datetime("2020-01-22")
end_date = pd.to_datetime(pd.datetime.now().date())

C:\Users\gdanish\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  


In [306]:
# SELECTING BETWEEN THE START AND END DATES
state_df = state_df.loc[start_date:end_date].reset_index()
state_df['date'] = state_df.date.astype(str)

In [307]:
state_df.date.max()

'2020-04-27'

In [308]:
state_df

,date,confirmed_incr,deaths_incr,recovered_incr,confirmed,deaths,recovered,deaths_incr_rate,confirmed_incr_rate,recovered_incr_rate
0,2020-03-14,14,0,0,14,0,0,0.0%,0.0%,0.0%
1,2020-03-15,18,0,0,32,0,0,0.0%,129.0%,0.0%
2,2020-03-16,6,0,0,38,0,0,0.0%,19.0%,0.0%
3,2020-03-17,3,1,0,41,1,0,0.0%,8.0%,0.0%
4,2020-03-18,3,0,0,44,1,0,0.0%,7.0%,0.0%
5,2020-03-19,4,0,0,48,1,0,0.0%,9.0%,0.0%
6,2020-03-20,4,0,0,52,1,0,0.0%,8.0%,0.0%
7,2020-03-21,12,0,0,64,1,0,0.0%,23.0%,0.0%
8,2020-03-22,10,1,0,74,2,0,100.0%,16.0%,0.0%
9,2020-03-23,23,0,0,97,2,0,0.0%,31.0%,0.0%
